## Import utilities

In [8]:
# import import_ipynb
from my_library import *
import pandas as pd
import numpy as np

# K-NN algorithm implementation

In [14]:
def knn_algo(df: pd.DataFrame, test_data: pd.DataFrame, options: dict = {}, ignore_columns: list = []):
    ignore_columns.append('id')
    ignore_columns.append('class')
    jaccard = True
    for col in df.columns:
        if col in ignore_columns:
            continue
        if not df[col].isin([0, 1]).all():
            jaccard = False
            break

    if not jaccard:
        # normalize data
        normalize_data(df, test_data, ignore_columns)

        print(f'test_data :')
        print(test_data)

        print(f'df :')
        print(df, end='\n\n')

        # calculate distance
        df['distance'] = df.apply(
            lambda row: distance(row, test_data.loc[0], columns=test_data.columns, options=options), axis=1)
    else:
        # calculate distance
        df['distance'] = df.apply(lambda row: jaccard_coefficient(row.values, test_data.values[0]), axis=1)

    # sort by distance
    df = df.sort_values(by=['distance'])

    print(df)

### Test the algorithm with the data

In [15]:
import numpy as np
import pandas as pd

data = pd.read_csv('testText.csv')
# test_data = {
#     'poids': [43],
#     'cheveux': ['brun'],
#     'taile': [1],
#     'vegetarien': [1]
# }
test_data = {
    'c1':[0],
    'c2':[0],
    'c3':[0],
    'c4':[0],
    'c5':[0],
    'c6':[0],
    'c7':[1],
    'c8':[0],
    'c9':[0],
    'c10':[1],
    'c11':[0]
}
test_data = pd.DataFrame(test_data)

knn_algo(data, test_data=test_data, ignore_columns=['name'])


   id  c1  c2  c3  c4  c5  c6  c7  c8  c9  c10  c11      class  distance
4   5   0   0   0   0   0   1   0   0   1    0    0  Politique  0.000000
3   4   0   0   0   0   0   0   0   0   1    1    1  Politique  0.666667
0   1   1   1   0   0   0   1   0   0   0    0    0      Sport  0.800000
1   2   1   0   0   1   0   0   0   1   0    0    0      Sport  1.000000
2   3   1   0   1   0   0   0   0   0   0    0    0      Sport  1.000000
5   6   0   0   0   0   0   0   1   0   0    1    0  Politique  1.000000
